In [5]:
import gensim,logging
import os
import sys
from os import listdir,makedirs
from os.path import isfile, join,exists
import glob
import json
import spacy
import re
from nltk.tokenize import sent_tokenize, word_tokenize
from collections import defaultdict,Counter
from nltk.corpus import stopwords
import string
import nltk
import numpy as np
import codecs
import random
from tqdm import tqdm

In [6]:
# get data
# Full cofea dataset
data_dir = join('data')
data_files = [filename for filename in glob.iglob(join(data_dir,'**/*.json'), recursive=True)]
full_data = []
filter_year = [1750,1810]
filter_source = ['Evans Early American Imprints','HeinOnline','National Archives Founders Online']

"""
for file in data_files: # adjust index slice here for the desired source
    if 'evans' in file or 'hein' in file or 'founders' in file:
        with open(file, encoding = 'utf-8') as f:
            #full_data = full_data + f.read().splitlines()
            # if using json
            docs = json.load(f)
            for doc in docs:
                if 'decade' in doc.keys():
                    if filter_year[0] <= int(doc['decade']) <= filter_year[1]:
                        full_data.append(doc['body'])
"""

        
save_name = 'COFEA'
with open(join('data','phrases.txt'), encoding = 'utf-8') as f:
    phrases = f.read().splitlines()
out_dir = join('data','preprocessed')

In [7]:
docs = []
# clean up cofea
for infile in data_files:
    with open(infile) as f:
        data = json.load(f)        
    print(infile, len(data), type(data))    
    for d in data:
        # fix one document with many problems (http://founders.archives.gov/documents/Jefferson/99-01-02-9951)
        if d['id'] == 'fndrs.jefferson.99-01-02-9951':            
            d['author'] == 'Wright, Robert'
            d['year'] = 1809
            d['decade'] = 1800
            d['collection'] = 'Jefferson Papers'
        # fix another that has year and decade listed as 2000:
        elif d['id'] == 'fndrs.jefferson.01-42-02-0442-0002':
            d['year'] = 1804
            d['decade'] = 1800
        # fix one document that clearly has the wrong year/decade (17626/17606)
        elif d['id'] == 'evans.N07112':
            d['year'] = 1762
            d['decade'] = 1760
        # fix years and decades for Elliot's debates (many listed as "2018")
        elif d['source'] == "Elliot's Debates":
            if 'year' in d and int(d['year']) == 2018:
                d.pop('year')
            d['decade'] = 1780
        # convert all years and decades to ints
        if 'year' in d:
            d['year'] = int(d['year'])
        if 'decade' in d:
            d['decade'] = int(d['decade'])
        if d['title'] != 'Editorial Note':
            docs.append(d)

data/statutes_output.json 481 <class 'list'>
data/hein_output.json 285 <class 'list'>
data/founders_output.json 181309 <class 'list'>
data/farrands_output.json 847 <class 'list'>
data/elliots_output.json 652 <class 'list'>
data/evans_output.json 4977 <class 'list'>


In [8]:
for doc in docs:
    if 'source' in doc.keys() and 'decade' in doc.keys():
        if doc['source'] in filter_source and (filter_year[0] <= doc['decade'] <= filter_year[1]):
            full_data.append(doc['body'])
        

In [9]:
nlp = spacy.load("en_core_web_sm")
tokenizer = nlp.tokenizer
stop = stopwords.words('english')

In [10]:
def clean_doc(doc,p_check=False):
    # break documents up into sentences and make sure each token is separated by just one space
    doc = doc.strip()
    doc = re.sub('(\\n|\\t|\\s)+'," ",doc)
    if p_check:
        for p in phrases:
            psub = p.replace(' ','')
            doc = doc.replace(p,psub)
    sents = sent_tokenize(doc)
    sents = [ ' '.join([y.text for y in tokenizer(x)]).lower() for x in sents] 
    return sents

In [11]:
def run_and_save(fname):
    
    #train model
    model = gensim.models.Word2Vec( alpha=0.025, window=4,vector_size=300, min_count=10, workers=12, sg=1, hs=0, negative=5)
    model.build_vocab(gensim.models.word2vec.LineSentence(fname+'.txt'))
    model.train(gensim.models.word2vec.LineSentence(fname+'.txt'), total_examples=model.corpus_count, epochs=5)
    model.wv.save_word2vec_format(fname+ '.tmp') # the gonen code takes the data in this form
    # save .wv.npy and .vocab
    vec = []
    w = codecs.open(fname + '.vocab', 'w', encoding='utf-8')
    vocab_size, embed_dim = None, None
    with codecs.open(fname + '.tmp', 'r', encoding='utf-8', errors='ignore') as r:
        for line in r:
            items = line.strip().split()
            if not vocab_size:
                assert(len(items) == 2)
                vocab_size, embed_dim = int(items[0]), int(items[1])
            else:
                assert(len(items) == embed_dim + 1)
                vec.append([float(item) for item in items[1:]])
                w.write('%s\n'%items[0])
    w.close()
    vec = np.array(vec, dtype=np.float)
    assert(vec.shape[0] == vocab_size)
    assert(vec.shape[1] == embed_dim)
    np.save(fname + '.wv.npy', vec)
    print('saved %s.wv.npy'%fname)
    print('saved %s.vocab'%fname)
    os.remove(fname + '.tmp')
    

In [14]:
out_name = save_name + '.txt' #change name based on source
# Save to text file so it can be efficiently used by word2vec
with open(join(out_dir,out_name), 'w',encoding = 'utf-8') as filehandle:
    for d in tqdm(full_data):
        # clean up the docs for processing
        sents = [clean_doc(d,True)]
        for s in sents:
            filehandle.write('%s\n' % s)

100%|██████████████████████████████████| 172584/172584 [18:44<00:00, 153.51it/s]


In [ ]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
out_name = join(out_dir,save_name) 
run_and_save(out_name)

2022-05-22 17:24:03,080 : INFO : Word2Vec lifecycle event {'params': 'Word2Vec<vocab=0, vector_size=300, alpha=0.025>', 'datetime': '2022-05-22T17:24:03.079950', 'gensim': '4.2.0', 'python': '3.9.7 | packaged by conda-forge | (default, Sep 29 2021, 19:23:11) \n[GCC 9.4.0]', 'platform': 'Linux-5.4.0-105-generic-x86_64-with-glibc2.31', 'event': 'created'}
2022-05-22 17:24:03,081 : INFO : collecting all words and their counts
2022-05-22 17:24:03,082 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-05-22 17:24:04,618 : INFO : PROGRESS: at sentence #10000, processed 4745780 words, keeping 82091 word types
2022-05-22 17:24:06,362 : INFO : PROGRESS: at sentence #20000, processed 10022701 words, keeping 138176 word types
2022-05-22 17:24:08,227 : INFO : PROGRESS: at sentence #30000, processed 15685926 words, keeping 185714 word types
2022-05-22 17:24:09,252 : INFO : PROGRESS: at sentence #40000, processed 18723053 words, keeping 218903 word types
2022-05-22 17:24

2022-05-22 17:25:24,867 : INFO : EPOCH 0 - PROGRESS: at 10.28% examples, 198959 words/s, in_qsize 23, out_qsize 0
2022-05-22 17:25:25,873 : INFO : EPOCH 0 - PROGRESS: at 10.36% examples, 198780 words/s, in_qsize 23, out_qsize 1
2022-05-22 17:25:26,896 : INFO : EPOCH 0 - PROGRESS: at 10.44% examples, 198399 words/s, in_qsize 24, out_qsize 2
2022-05-22 17:25:27,901 : INFO : EPOCH 0 - PROGRESS: at 10.71% examples, 199283 words/s, in_qsize 24, out_qsize 0
2022-05-22 17:25:28,909 : INFO : EPOCH 0 - PROGRESS: at 10.95% examples, 199223 words/s, in_qsize 24, out_qsize 0
2022-05-22 17:25:29,936 : INFO : EPOCH 0 - PROGRESS: at 11.23% examples, 199019 words/s, in_qsize 23, out_qsize 0
2022-05-22 17:25:30,989 : INFO : EPOCH 0 - PROGRESS: at 11.55% examples, 199152 words/s, in_qsize 23, out_qsize 0
2022-05-22 17:25:31,989 : INFO : EPOCH 0 - PROGRESS: at 11.75% examples, 199187 words/s, in_qsize 23, out_qsize 0
2022-05-22 17:25:33,003 : INFO : EPOCH 0 - PROGRESS: at 11.97% examples, 199110 words/s,

2022-05-22 17:26:39,695 : INFO : EPOCH 0 - PROGRESS: at 37.75% examples, 200601 words/s, in_qsize 23, out_qsize 0
2022-05-22 17:26:40,765 : INFO : EPOCH 0 - PROGRESS: at 38.09% examples, 200586 words/s, in_qsize 23, out_qsize 0
2022-05-22 17:26:41,810 : INFO : EPOCH 0 - PROGRESS: at 38.40% examples, 200543 words/s, in_qsize 23, out_qsize 0
2022-05-22 17:26:42,831 : INFO : EPOCH 0 - PROGRESS: at 38.77% examples, 200758 words/s, in_qsize 22, out_qsize 0
2022-05-22 17:26:43,872 : INFO : EPOCH 0 - PROGRESS: at 39.11% examples, 200479 words/s, in_qsize 23, out_qsize 0
2022-05-22 17:26:44,875 : INFO : EPOCH 0 - PROGRESS: at 39.47% examples, 200500 words/s, in_qsize 23, out_qsize 0
2022-05-22 17:26:45,887 : INFO : EPOCH 0 - PROGRESS: at 39.90% examples, 200657 words/s, in_qsize 23, out_qsize 0
2022-05-22 17:26:46,895 : INFO : EPOCH 0 - PROGRESS: at 40.35% examples, 200644 words/s, in_qsize 23, out_qsize 0
2022-05-22 17:26:47,895 : INFO : EPOCH 0 - PROGRESS: at 40.84% examples, 200679 words/s,

2022-05-22 17:27:53,919 : INFO : EPOCH 0 - PROGRESS: at 73.34% examples, 201150 words/s, in_qsize 24, out_qsize 0
2022-05-22 17:27:54,925 : INFO : EPOCH 0 - PROGRESS: at 73.76% examples, 201106 words/s, in_qsize 23, out_qsize 0
2022-05-22 17:27:55,964 : INFO : EPOCH 0 - PROGRESS: at 74.18% examples, 201032 words/s, in_qsize 22, out_qsize 1
2022-05-22 17:27:56,986 : INFO : EPOCH 0 - PROGRESS: at 74.68% examples, 201179 words/s, in_qsize 24, out_qsize 0
2022-05-22 17:27:58,046 : INFO : EPOCH 0 - PROGRESS: at 75.14% examples, 201121 words/s, in_qsize 23, out_qsize 0
2022-05-22 17:27:59,047 : INFO : EPOCH 0 - PROGRESS: at 75.58% examples, 201128 words/s, in_qsize 23, out_qsize 0
2022-05-22 17:28:00,099 : INFO : EPOCH 0 - PROGRESS: at 76.08% examples, 201160 words/s, in_qsize 23, out_qsize 0
2022-05-22 17:28:01,119 : INFO : EPOCH 0 - PROGRESS: at 76.49% examples, 201159 words/s, in_qsize 23, out_qsize 0
2022-05-22 17:28:02,212 : INFO : EPOCH 0 - PROGRESS: at 76.90% examples, 201102 words/s,

2022-05-22 17:29:08,249 : INFO : EPOCH 0 - PROGRESS: at 95.11% examples, 201212 words/s, in_qsize 23, out_qsize 0
2022-05-22 17:29:09,267 : INFO : EPOCH 0 - PROGRESS: at 95.13% examples, 201201 words/s, in_qsize 23, out_qsize 0
2022-05-22 17:29:10,388 : INFO : EPOCH 0 - PROGRESS: at 95.15% examples, 201110 words/s, in_qsize 23, out_qsize 0
2022-05-22 17:29:11,392 : INFO : EPOCH 0 - PROGRESS: at 95.17% examples, 201123 words/s, in_qsize 22, out_qsize 1
2022-05-22 17:29:12,439 : INFO : EPOCH 0 - PROGRESS: at 95.19% examples, 201140 words/s, in_qsize 23, out_qsize 0
2022-05-22 17:29:13,483 : INFO : EPOCH 0 - PROGRESS: at 95.21% examples, 201128 words/s, in_qsize 23, out_qsize 0
2022-05-22 17:29:14,511 : INFO : EPOCH 0 - PROGRESS: at 95.23% examples, 201163 words/s, in_qsize 23, out_qsize 1
2022-05-22 17:29:15,513 : INFO : EPOCH 0 - PROGRESS: at 95.25% examples, 201152 words/s, in_qsize 24, out_qsize 0
2022-05-22 17:29:16,547 : INFO : EPOCH 0 - PROGRESS: at 95.27% examples, 201092 words/s,

...

2022-05-22 18:00:41,589 : INFO : EPOCH 4 - PROGRESS: at 39.71% examples, 202394 words/s, in_qsize 23, out_qsize 0
2022-05-22 18:00:42,626 : INFO : EPOCH 4 - PROGRESS: at 40.08% examples, 202212 words/s, in_qsize 24, out_qsize 1
2022-05-22 18:00:43,646 : INFO : EPOCH 4 - PROGRESS: at 40.54% examples, 202135 words/s, in_qsize 22, out_qsize 1
2022-05-22 18:00:44,662 : INFO : EPOCH 4 - PROGRESS: at 41.08% examples, 202333 words/s, in_qsize 24, out_qsize 0
2022-05-22 18:00:45,769 : INFO : EPOCH 4 - PROGRESS: at 41.50% examples, 202138 words/s, in_qsize 23, out_qsize 0
2022-05-22 18:00:46,804 : INFO : EPOCH 4 - PROGRESS: at 41.97% examples, 202333 words/s, in_qsize 23, out_qsize 0
2022-05-22 18:00:47,815 : INFO : EPOCH 4 - PROGRESS: at 42.09% examples, 202361 words/s, in_qsize 24, out_qsize 0
2022-05-22 18:00:48,837 : INFO : EPOCH 4 - PROGRESS: at 42.45% examples, 202224 words/s, in_qsize 23, out_qsize 0
2022-05-22 18:00:49,852 : INFO : EPOCH 4 - PROGRESS: at 42.79% examples, 202241 words/s,

2022-05-22 18:01:56,345 : INFO : EPOCH 4 - PROGRESS: at 76.18% examples, 202455 words/s, in_qsize 23, out_qsize 0
2022-05-22 18:01:57,351 : INFO : EPOCH 4 - PROGRESS: at 76.59% examples, 202499 words/s, in_qsize 24, out_qsize 1
2022-05-22 18:01:58,363 : INFO : EPOCH 4 - PROGRESS: at 77.03% examples, 202521 words/s, in_qsize 23, out_qsize 0
2022-05-22 18:01:59,479 : INFO : EPOCH 4 - PROGRESS: at 77.53% examples, 202443 words/s, in_qsize 23, out_qsize 0
2022-05-22 18:02:00,528 : INFO : EPOCH 4 - PROGRESS: at 78.10% examples, 202559 words/s, in_qsize 23, out_qsize 0
2022-05-22 18:02:01,661 : INFO : EPOCH 4 - PROGRESS: at 78.64% examples, 202487 words/s, in_qsize 23, out_qsize 0
2022-05-22 18:02:02,665 : INFO : EPOCH 4 - PROGRESS: at 79.16% examples, 202524 words/s, in_qsize 23, out_qsize 0
2022-05-22 18:02:03,668 : INFO : EPOCH 4 - PROGRESS: at 79.69% examples, 202573 words/s, in_qsize 23, out_qsize 0
2022-05-22 18:02:04,674 : INFO : EPOCH 4 - PROGRESS: at 80.30% examples, 202504 words/s,

2022-05-22 18:03:10,931 : INFO : EPOCH 4 - PROGRESS: at 95.25% examples, 202382 words/s, in_qsize 23, out_qsize 0
2022-05-22 18:03:11,934 : INFO : EPOCH 4 - PROGRESS: at 95.27% examples, 202425 words/s, in_qsize 23, out_qsize 0
2022-05-22 18:03:12,947 : INFO : EPOCH 4 - PROGRESS: at 95.29% examples, 202424 words/s, in_qsize 23, out_qsize 0
2022-05-22 18:03:13,959 : INFO : EPOCH 4 - PROGRESS: at 95.31% examples, 202366 words/s, in_qsize 23, out_qsize 0
2022-05-22 18:03:14,962 : INFO : EPOCH 4 - PROGRESS: at 95.33% examples, 202405 words/s, in_qsize 23, out_qsize 0
2022-05-22 18:03:15,972 : INFO : EPOCH 4 - PROGRESS: at 95.34% examples, 202358 words/s, in_qsize 23, out_qsize 0
2022-05-22 18:03:16,973 : INFO : EPOCH 4 - PROGRESS: at 95.37% examples, 202376 words/s, in_qsize 24, out_qsize 2
2022-05-22 18:03:18,032 : INFO : EPOCH 4 - PROGRESS: at 95.39% examples, 202325 words/s, in_qsize 23, out_qsize 0
2022-05-22 18:03:19,061 : INFO : EPOCH 4 - PROGRESS: at 95.41% examples, 202373 words/s,

2022-05-22 18:04:25,003 : INFO : EPOCH 4 - PROGRESS: at 96.74% examples, 202165 words/s, in_qsize 24, out_qsize 0
2022-05-22 18:04:26,059 : INFO : EPOCH 4 - PROGRESS: at 96.76% examples, 202144 words/s, in_qsize 23, out_qsize 0
2022-05-22 18:04:27,109 : INFO : EPOCH 4 - PROGRESS: at 96.78% examples, 202169 words/s, in_qsize 22, out_qsize 1
2022-05-22 18:04:28,159 : INFO : EPOCH 4 - PROGRESS: at 96.80% examples, 202167 words/s, in_qsize 23, out_qsize 0
2022-05-22 18:04:29,211 : INFO : EPOCH 4 - PROGRESS: at 96.82% examples, 202181 words/s, in_qsize 24, out_qsize 1
2022-05-22 18:04:30,217 : INFO : EPOCH 4 - PROGRESS: at 96.84% examples, 202165 words/s, in_qsize 23, out_qsize 0
2022-05-22 18:04:31,303 : INFO : EPOCH 4 - PROGRESS: at 96.86% examples, 202127 words/s, in_qsize 23, out_qsize 2
2022-05-22 18:04:32,333 : INFO : EPOCH 4 - PROGRESS: at 96.88% examples, 202150 words/s, in_qsize 23, out_qsize 0
2022-05-22 18:04:33,387 : INFO : EPOCH 4 - PROGRESS: at 96.89% examples, 202130 words/s,

2022-05-22 18:05:40,316 : INFO : EPOCH 4 - PROGRESS: at 98.11% examples, 202007 words/s, in_qsize 23, out_qsize 3
2022-05-22 18:05:41,532 : INFO : EPOCH 4 - PROGRESS: at 98.13% examples, 202004 words/s, in_qsize 23, out_qsize 0
2022-05-22 18:05:42,552 : INFO : EPOCH 4 - PROGRESS: at 98.15% examples, 202041 words/s, in_qsize 23, out_qsize 0
2022-05-22 18:05:43,715 : INFO : EPOCH 4 - PROGRESS: at 98.17% examples, 201940 words/s, in_qsize 23, out_qsize 0
2022-05-22 18:05:44,819 : INFO : EPOCH 4 - PROGRESS: at 98.19% examples, 202018 words/s, in_qsize 23, out_qsize 0
2022-05-22 18:05:45,861 : INFO : EPOCH 4 - PROGRESS: at 98.21% examples, 201972 words/s, in_qsize 23, out_qsize 2
2022-05-22 18:05:46,867 : INFO : EPOCH 4 - PROGRESS: at 98.23% examples, 201986 words/s, in_qsize 23, out_qsize 0
2022-05-22 18:05:47,948 : INFO : EPOCH 4 - PROGRESS: at 98.24% examples, 201967 words/s, in_qsize 23, out_qsize 0
2022-05-22 18:05:48,967 : INFO : EPOCH 4 - PROGRESS: at 98.26% examples, 201987 words/s,

In [16]:
'h'

'h'